# Misspelling detection and correction

**NOTE**: This notebook depends upon the the Retrotech dataset. If you have any issues, please rerun the [Setting up the Retrotech Dataset](../ch4/1.ch4-setting-up-the-retrotech-dataset.ipynb) notebook.

In [196]:
import sys
sys.path.append('..')
import json
from aips import *
import pandas as pd
import nltk
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import numpy as np
import re
nltk.download('stopwords')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("aips-ch6").getOrCreate()

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Listing 6.12

In [198]:
collection="products"
query="moden"

request = {
    "params": {
        "q.op": "and",
        "rows": 0,
        "indent": "on"
    },
    "query": query,
}

search_results = requests.post(solr_url + collection + "/spell", json=request).json()
print(json.dumps(search_results["spellcheck"]["collations"], indent=4))

[
    "collation",
    {
        "collationQuery": "modern",
        "hits": 42,
        "misspellingsAndCorrections": [
            "moden",
            "modern"
        ]
    },
    "collation",
    {
        "collationQuery": "model",
        "hits": 40,
        "misspellingsAndCorrections": [
            "moden",
            "model"
        ]
    },
    "collation",
    {
        "collationQuery": "modem",
        "hits": 29,
        "misspellingsAndCorrections": [
            "moden",
            "modem"
        ]
    },
    "collation",
    {
        "collationQuery": "modena",
        "hits": 1,
        "misspellingsAndCorrections": [
            "moden",
            "modena"
        ]
    },
    "collation",
    {
        "collationQuery": "modes",
        "hits": 1,
        "misspellingsAndCorrections": [
            "moden",
            "modes"
        ]
    }
]


In [160]:
#use the real signals
signals_collection="signals"
signals_opts={"zkhost": "aips-zk", "collection": signals_collection}
df = spark.read.format("solr").options(**signals_opts).load()
df.createOrReplaceTempView("signals")

## Listing 6.13

In [230]:
### Create user-searchs table each raw represent one search query.
query_signals = spark.sql("""
  select lower(trim(searches.target)) as keyword, searches.user as user 
  from signals as searches where searches.type='query'
  group by keyword, user"""
).collect()

### Step 1: Tokenize queries and count word frequencies. 
Check word frequency distribution quantiles. The quantile will help decide cut off point for potential misspellings and corrections. 

## Listing 6.14

In [162]:
stop_words = set(stopwords.words('english'))
word_list = defaultdict(int)

#for query in signal_sample["query_s"]:
for row in query_signals:
    query = row["keyword"]
    tokenizer = RegexpTokenizer(r'\w+') 
    tokens   = tokenizer.tokenize(query)
    
    for token in tokens:
        if token not in stop_words and len(token) > 3 and not token.isdigit():  #drop stopwords and digit only tokens
            # and only consider token length > 3, since hard to judge whether a very short token is misspelled or not
            word_list[token] += 1

## Listing 6.15

In [232]:
quantiles_to_check = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
quantile_values = np.quantile(np.array(list(word_list.values())), quantiles_to_check)
quantiles = dict(zip(quantiles_to_check, quantile_values))
quantiles

{0.1: 5.0,
 0.2: 6.0,
 0.3: 7.0,
 0.4: 8.0,
 0.5: 10.0,
 0.6: 13.0,
 0.7: 18.0,
 0.8: 30.0,
 0.9: 113.0}

### Step 2: compute metadata needed for word matching. 
consider word with low count as misspelling condidates, with high count as correctly spelled candidates. 

## Listing 6.16

In [233]:
misspell_candidates = []
correction_candidates = []
misspell_counts = []
correction_counts = []
misspell_length = []
correction_length = []
misspell_initial = []
correction_initial = []

for k, v in word_list.items():
    if v <= quantiles[0.2] : #if v == 1:  # this number based on quantile analysis and the data set, more-likely with user-behvaiour data set to be 1
        misspell_candidates.append(k)
        misspell_counts.append(v)
        misspell_length.append(len(k))
        misspell_initial.append(k[0])
    if v >= quantiles[0.8]:
        correction_candidates.append(k)
        correction_counts.append(v)
        correction_length.append(len(k))
        correction_initial.append(k[0])

## Listing 6.17

In [234]:
misspell_candidates_df = pd.DataFrame({
    "misspell":misspell_candidates, 
    "misspell_counts":misspell_counts, 
    "misspell_length":misspell_length,
    "initial":misspell_initial})

correction_candidates_df = pd.DataFrame({
    "correction":correction_candidates, 
    "correction_counts":correction_counts, 
    "correction_length":correction_length,
    "initial":correction_initial})

In [235]:
#Show Results:
misspell_candidates_df.head(10)

,misspell,misspell_counts,misspell_length,initial
0,otterbox iphone cases,6,21,o
1,tripod manfrotto,5,16,t
2,singin in the rain,6,18,s
3,canon powershot elph,6,20,c
4,fight stick,5,11,f
5,htc evo view,5,12,h
6,sx130,6,5,s
7,klipsch image,6,13,k
8,tablet case 7,5,13,t
9,borderlands 2,5,13,b


### Step 3: Find potential matches 
based on edit distance and whether word initial is the same or not. 

## Lsting 6.18

In [236]:
def good_match(len1, len2, edit_dist): #allow longer words have more edit distance
    match = 0
    min_length = min(len1, len2)
    if min_length < 8:
        if edit_dist == 1: match = 1
    elif min_length < 11:
        if edit_dist <= 2: match = 1
    else:
        if edit_dist == 3: match = 1
    return match

## Listing 6.19

In [237]:
matches_candidates = pd.merge(misspell_candidates_df, correction_candidates_df, on="initial")
#join missepll list with correction list based on whether they share the same initials to reduce matching time. 
matches_candidates["edit_dist"] = matches_candidates.apply(lambda row: nltk.edit_distance(row.misspell,row.correction), axis=1)
matches_candidates["good_match"] = matches_candidates.apply(lambda row: good_match(row.misspell_length, row.correction_length, row.edit_dist),axis=1)

In [238]:
matches = matches_candidates[matches_candidates["good_match"] == 1].drop(["initial","good_match"],axis=1)

### Step 4: rank potential matched corrections 
based on edit distance and correction word frequency. shorter edit distance and higher word count will be prefered. only the top one correction is selected for final matching. 

In [239]:
#matches.sort_values(by=['misspell', 'edit_dist', 'correction_counts'], ascending=[True, True, False])
matches_final = matches.groupby('misspell').first().reset_index()

In [240]:
#Show Results:
matches_final.sort_values(by=['correction_counts'], ascending=[False])[["misspell", "correction", "misspell_counts", "correction_counts", "edit_dist"]].head(20)

,misspell,correction,misspell_counts,correction_counts,edit_dist
181,ipad.,ipad,6,7749,1
154,hp touchpad 32,hp touchpad,5,7144,3
155,hp toucpad,hp touchpad,6,7144,1
153,hp tochpad,hp touchpad,6,7144,1
190,iphone s4,iphone 4s,5,4642,2
193,iphone4 s,iphone 4s,5,4642,2
194,iphones 4s,iphone 4s,5,4642,1
412,touchpaf,touchpad,5,4019,1
406,tochpad,touchpad,6,4019,1
407,toichpad,touchpad,6,4019,1


## Listing 6.20

In [ ]:
## Alternative - don't tokenize into individual keywords
stop_words = set(stopwords.words('english'))
word_list = defaultdict(int)

#for query in signal_sample["query_s"]:
for row in query_signals:
    query = row["keyword"]

    if query not in stop_words and len(query) > 3 and not query.isdigit():  
        word_list[query] += 1
        
#run Listing 16.12-16.15 again...